In [1]:
import pandas as pd
import numpy as np
from collections import Counter

# dataset is accessible at https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt (https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz)
df = pd.read_csv('amazon_reviews_us_Digital_Software_v1_00.tsv', sep='\t', header=None, on_bad_lines='skip')

print(df.head())

            0            1               2           3               4   \
0  marketplace  customer_id       review_id  product_id  product_parent   
1           US     17747349  R2EI7QLPK4LF7U  B00U7LCE6A       106182406   
2           US     10956619  R1W5OMFK1Q3I3O  B00HRJMOM4       162269768   
3           US     13132245   RPZWSYWRP92GI  B00P31G9PQ       831433899   
4           US     35717248  R2WQWM04XHD9US  B00FGDEPDY       991059534   

                                           5                 6            7   \
0                               product_title  product_category  star_rating   
1                    CCleaner Free [Download]  Digital_Software            4   
2          ResumeMaker Professional Deluxe 18  Digital_Software            3   
3                   Amazon Drive Desktop [PC]  Digital_Software            1   
4  Norton Internet Security 1 User 3 Licenses  Digital_Software            5   

              8            9     10                 11              

C:\Users\Frederico\AppData\Local\Temp\ipykernel_9728\643836554.py:6: DtypeWarning: Columns (1,4,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('amazon_reviews_us_Digital_Software_v1_00.tsv', sep='\t', header=None, on_bad_lines='skip')


In [2]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101837 entries, 0 to 101836
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       101837 non-null  object
 1   1       101837 non-null  object
 2   2       101837 non-null  object
 3   3       101837 non-null  object
 4   4       101837 non-null  object
 5   5       101837 non-null  object
 6   6       101837 non-null  object
 7   7       101837 non-null  object
 8   8       101837 non-null  object
 9   9       101837 non-null  object
 10  10      101837 non-null  object
 11  11      101837 non-null  object
 12  12      101837 non-null  object
 13  13      101837 non-null  object
 14  14      101832 non-null  object
dtypes: object(15)
memory usage: 11.7+ MB


In [3]:
df.columns = ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 
            'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']

print(df.head())

   marketplace  customer_id       review_id  product_id  product_parent  \
0  marketplace  customer_id       review_id  product_id  product_parent   
1           US     17747349  R2EI7QLPK4LF7U  B00U7LCE6A       106182406   
2           US     10956619  R1W5OMFK1Q3I3O  B00HRJMOM4       162269768   
3           US     13132245   RPZWSYWRP92GI  B00P31G9PQ       831433899   
4           US     35717248  R2WQWM04XHD9US  B00FGDEPDY       991059534   

                                product_title  product_category  star_rating  \
0                               product_title  product_category  star_rating   
1                    CCleaner Free [Download]  Digital_Software            4   
2          ResumeMaker Professional Deluxe 18  Digital_Software            3   
3                   Amazon Drive Desktop [PC]  Digital_Software            1   
4  Norton Internet Security 1 User 3 Licenses  Digital_Software            5   

   helpful_votes  total_votes  vine  verified_purchase     review_he

In [16]:
print(df['review_body'][1:10])
print(df['star_rating'][1:10])  

1                                       So far so good
2                        Needs a little more work.....
3                                       Please cancel.
4                                   Works as Expected!
5    I've had Webroot for a few years. It expired a...
6    EXCELLENT software !!!!!  Don't need to do any...
7    The variations created by Win10 induced this p...
8    Horrible!  Would not upgrade previous version ...
9                                      Waste of time .
Name: review_body, dtype: object
1    4
2    3
3    1
4    5
5    4
6    5
7    1
8    1
9    1
Name: star_rating, dtype: object


In [4]:
# from sklearn.model_selection import train_test_split 

# train, test = train_test_split(df, test_size=0.30, random_state=42)


## Sentiment analysis

In [17]:
all_reviews = df['review_body'][1:]
all_ratings = df['star_rating'][1:]

In [18]:
# reviews_train = list()
# for review in train['review_body']:
#     reviews_train.append(review)

# for review in test['review_body']:
#     reviews_test.append(review)

# print(reviews_train[:10])
# print(reviews_test[:10])

reviews = list()

for review in all_reviews:
    reviews.append(review)

print(reviews[:10])

['So far so good', 'Needs a little more work.....', 'Please cancel.', 'Works as Expected!', "I've had Webroot for a few years. It expired and I decided to purchase a renewal on Amazon. I went through hell trying to uninstall the expired version in order to install the new.  I called Webroot and had their representative remote into my computer at his request. He was clueless as a bad joke and consumed 29 minutes and 57 seconds of my time forever.  He initially told me it wasn't compatible with Windows 10, but I finally managed to convince him that it is indeed compatible with Windows 10 as it was working on my computer before it expired and also I showed him a review on Amazon to convince him that it works on Windows 10. Finally, he offered to connect me with a senior consultant for over 100 dollars. I declined and told him I'd fix the issue myself. This guy was less helpful than a severed limb.  After spending some time on Google, the issue is now fixed. Webroot should just get rid of 

In [19]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# corpus_train = []
# ps = PorterStemmer()
# sw = set(stopwords.words('english'))
# for i in range(0, len(reviews_train)):
#     # get review and remove non alpha chars
#     review = re.sub('[^a-zA-Z]', ' ', reviews_train[i])
#     # to lower-case
#     review = review.lower()
#     # split into tokens, apply stemming and remove stop words
#     review = ' '.join([ps.stem(w) for w in review.split() if w not in sw])
#     corpus_train.append(review)

# corpus_test = []
# for i in range(0, len(reviews_test)):
#     # get review and remove non alpha chars
#     review = re.sub('[^a-zA-Z]', ' ', reviews_test[i])
#     # to lower-case
#     review = review.lower()
#     # split into tokens, apply stemming and remove stop words
#     review = ' '.join([ps.stem(w) for w in review.split() if w not in sw])
#     corpus_test.append(review)   

# print(corpus_train[:10])
# print(corpus_test[:10])

corpus = list()
ps = PorterStemmer()
sw = set(stopwords.words('english'))
for i in range(0, len(reviews)):
    # get review and remove non alpha chars
    review = re.sub('[^a-zA-Z]', ' ', reviews[i])
    # to lower-case
    review = review.lower()
    # split into tokens, apply stemming and remove stop words
    review = ' '.join([ps.stem(w) for w in review.split() if w not in sw])
    corpus.append(review)

print(corpus[:10])


['far good', 'need littl work', 'pleas cancel', 'work expect', 'webroot year expir decid purchas renew amazon went hell tri uninstal expir version order instal new call webroot repres remot comput request clueless bad joke consum minut second time forev initi told compat window final manag convinc inde compat window work comput expir also show review amazon convinc work window final offer connect senior consult dollar declin told fix issu guy less help sever limb spend time googl issu fix webroot get rid custom servic pay googl index much help info dedic custom servic offer far softwar think scan fast slow comput hope like peopl includ expert effect remov prevent malwar year ago extens research found among best year ago thing fluid malwar kingdom experienc issu instal advic bother uninstal old version rather launch webroot click account right side copi past product key area say enter new keycod click activ save numer pain hour tri get thing work', 'excel softwar need anyth set automat 

In [20]:
# ratings_train = list()
# ratings_test = list()
# for rating in train['star_rating']:
#     ratings_train.append(rating)

# for rating in test['star_rating']:
#     ratings_test.append(rating)

# print(ratings_train[:10])
# print(ratings_test[:10])

#print(all_ratings[:10])

1     4
2     3
3     1
4     5
5     4
6     5
7     1
8     1
9     1
10    5
Name: star_rating, dtype: object


In [ ]:
# #word embeddings
# import gensim
# from gensim.models import Word2Vec,KeyedVectors

# model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz',binary=True,limit=100000)


In [ ]:
# import numpy as np

# def text_to_vector(embeddings, text, sequence_len):
    
#     # split text into tokens
#     tokens = text.split()
    
#     # convert tokens to embedding vectors, up to sequence_len tokens
#     vec = []
#     n = 0
#     i = 0
#     while i < len(tokens) and n < sequence_len:   # while there are tokens and did not reach desired sequence length
#         try:
#             vec.extend(embeddings.get_vector(tokens[i]))
#             n += 1
#         except KeyError:
#             True   # simply ignore out-of-vocabulary tokens
#         finally:
#             i += 1
    
#     # add blanks up to sequence_len, if needed
#     for j in range(sequence_len - n):
#         vec.extend(np.zeros(embeddings.vector_size,))
    
#     return vec

In [ ]:
# embeddings_corpus = []
# for c in corpus:
#     embeddings_corpus.append(text_to_vector(model, c, 10))

# X = np.array(embeddings_corpus)
# y = all_ratings

# print(X.shape, y.shape)

In [21]:
from sklearn.model_selection import train_test_split 

X_train, X_test = train_test_split(corpus, test_size=0.30, random_state=42)
y_train, y_test = train_test_split(all_ratings, test_size=0.30, random_state=42)

print(X_train[:10])
print(y_train[:10])

['bewar download includ malwar', 'husband yet tell like sorri use seem pretti cool easi use', 'microsoft need fix bug outlook found far probabl sure', 'easi purchas download smooth hesit download never problem nice product within coupl minut purchas nice', 'wast money read comment first would purchas yuck', '', 'use avast long time think good everyon made web experi secur viru malwar worri free strongli recommend softwar experi', 'use tt ten year unfortun year program simpli work use window googl chrome browser troubl download updat thought okay realiz congress wait last minut come back went ahead entir tax return went back tri download updat download stall minut tri sever time alway result tri simpli file return without updat best idea pretti simpl return program allow e file although cours took electron file state return intuit clearli know problem superpatch websit download manual unfortun thing happen tri download superpatch spent hour hold repeatedli attempt download tri contact t

In [22]:
import torch
from torch.utils.data import DataLoader, TensorDataset

#create Tensor Dataset
train_data=TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
test_data=TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))

#dataloader
batch_size=50
train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader=DataLoader(test_data, batch_size=batch_size, shuffle=True)

ValueError: too many dimensions 'str'

In [ ]:
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

In [ ]:
import torch.nn as nn

class SentimentalLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):    
        """
        Initialize the model by setting up the layers
        """
        super().__init__()
        self.output_size=output_size
        self.n_layers=n_layers
        self.hidden_dim=hidden_dim
        
        #Embedding and LSTM layers
        self.embedding=nn.Embedding(vocab_size, embedding_dim)
        self.lstm=nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        
        #dropout layer
        self.dropout=nn.Dropout(0.3)
        
        #Linear and sigmoid layer
        self.fc1=nn.Linear(hidden_dim, 64)
        self.fc2=nn.Linear(64, 16)
        self.fc3=nn.Linear(16,output_size)
        self.sigmoid=nn.Sigmoid()
        
    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size=x.size()
        
        #Embadding and LSTM output
        embedd=self.embedding(x)
        lstm_out, hidden=self.lstm(embedd, hidden)
        
        #stack up the lstm output
        lstm_out=lstm_out.contiguous().view(-1, self.hidden_dim)
        
        #dropout and fully connected layers
        out=self.dropout(lstm_out)
        out=self.fc1(out)
        out=self.dropout(out)
        out=self.fc2(out)
        out=self.dropout(out)
        out=self.fc3(out)
        sig_out=self.sigmoid(out)
        
        sig_out=sig_out.view(batch_size, -1)
        sig_out=sig_out[:, -1]
        
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        """Initialize Hidden STATE"""
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [ ]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentalLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(net)

In [ ]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

# training params

epochs = 3 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs=inputs.cuda()
            labels=labels.cuda()
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                inputs, labels = inputs.cuda(), labels.cuda()  
                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))